In [46]:
import os
import glob
import pandas
import pprint
import shutil

In [2]:
assembly_dirs = '/Users/mf019/bioinformatics/longread_GWAS/assemblies'

In [3]:
#v5_file = '../wrangled/longread_metadata_v5.csv'
v6_file = '../wrangled/longread_metadata_v6.csv'
v7_file = '../wrangled/longread_metadata_v7.csv'
renaming_table_for_jupyter_file  = '../lymeseq_original_sheets/LymeSeq_SampleTrack - Renaming Scheme Table for Jupyter (1).csv'
renaming_scheme_key = '../lymeseq_original_sheets/LymeSeq_SampleTrack - Renaming Scheme Key (1).csv'
repeated_seq_sheet  = '../lymeseq_original_sheets/LymeSeq_SampleTrack - Repeated Sequence Sheet.csv'
wgs_sheet = '../lymeseq_original_sheets/LymeSeq_SampleTrack - Whole Genome Sequenced Table (1).csv'

In [4]:
#v5_df = pandas.read_csv(v5_file)
v6_df = pandas.read_csv(v6_file)
jupyter_df = pandas.read_csv(renaming_table_for_jupyter_file)
renaming_key_df = pandas.read_csv(renaming_scheme_key)
repeated_seq_df = pandas.read_csv(repeated_seq_sheet)
wgs_sheet_df = pandas.read_csv(wgs_sheet)

In [5]:
v6_df['paired_sample'] = v6_df['paired_sample'].str[:-1:]

In [6]:
# now lets get lists for each assembly method/seq method

list_of_pacbio = glob.glob(f'{assembly_dirs}/pacbio/contigs/*.fasta')
list_of_hybrid = glob.glob(f'{assembly_dirs}/hybrid/contigs/*.fasta')
list_of_illumina = []
for filename in glob.glob(f'{assembly_dirs}/illumina/contigs/*.fasta'):
    list_of_illumina.append(filename.split('/')[-1].split('.')[0].replace('_200', ''))
print(f'pacbio: {len(list_of_pacbio)}')
print(f'hybrid: {len(list_of_hybrid)}')
print(f'illumina: {len(list_of_illumina)}')


pacbio: 20
hybrid: 47
illumina: 299


In [7]:
#now lets sample the metadata for pacbio and hybrid into two dataframes

pacbio_df = v6_df[v6_df['method'] == 'pacbio']
paired_samples_pb = list(pacbio_df['paired_sample'])
hybrid_df = v6_df[v6_df['method'] == 'hybrid']
paired_samples_hyb = list(hybrid_df['paired_sample'])



In [8]:
pacbio_df['paired_sample'] = pacbio_df['paired_sample'].fillna(pacbio_df['newname'])
hybrid_df['paired_sample'] = hybrid_df['paired_sample'].fillna(hybrid_df['newname'])

/var/folders/59/fs0fxd1x1wx1j_stw1ccjbcr0000gp/T/ipykernel_10263/1476110409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pacbio_df['paired_sample'] = pacbio_df['paired_sample'].fillna(pacbio_df['newname'])
/var/folders/59/fs0fxd1x1wx1j_stw1ccjbcr0000gp/T/ipykernel_10263/1476110409.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hybrid_df['paired_sample'] = hybrid_df['paired_sample'].fillna(hybrid_df['newname'])


In [9]:
blood_tissue_pairs_table_pacbio = pacbio_df.loc[pacbio_df['skin/blood_pairs'].notna(), ['Isolate', 'skin/blood_pairs']]
blood_tissue_pairs_table_hybrid = hybrid_df.loc[hybrid_df['skin/blood_pairs'].notna(), ['Isolate', 'skin/blood_pairs']]

In [10]:
bloodtissue_pairs_jupyter_df = jupyter_df[jupyter_df['Original_Name'].isin(blood_tissue_pairs_table_pacbio['skin/blood_pairs'])]
bloodtissue_pairs_jupyter_df

,Rename_A,Original_Name,Origin,Location,OspC_Type,RST_Type,OspC_Allele,MLST,Disseminated,BC + (Y/N),...,Source.1,Code,Genus,Species.1,FastQC_1,BBDuk,FastQC_2,Spades,BBMerge,sp_identification
1021,UNY1021,BL146,NY,NaN,K,2.0,NaN,3,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1044,UNY1044,BL252,NY,NaN,E,3.0,NaN,19,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
pacbio_paired = jupyter_df[jupyter_df['Rename_A'].isin(pacbio_df['paired_sample'])]
## HERE IS THAT MISSING SAMPLE!
missing_row = jupyter_df[jupyter_df['Original_Name'] == 'MR641']
missing_row_p = jupyter_df[jupyter_df['Original_Name'] == 'MR641P']
##
pacbio_paired = pandas.concat([pacbio_paired, (jupyter_df[jupyter_df['Rename_A'].isin(pacbio_paired['Rename_A']+ 'P')]),missing_row, missing_row_p])
pacbio_paired

,Rename_A,Original_Name,Origin,Location,OspC_Type,RST_Type,OspC_Allele,MLST,Disseminated,BC + (Y/N),...,Source.1,Code,Genus,Species.1,FastQC_1,BBDuk,FastQC_2,Spades,BBMerge,sp_identification
148,UNY149,B247,US,NY,K,2.0,NaN,3,D,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,UNY169,B348,US,NY,E,3.0,ospC_AT5,19,D,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,UNY172,B356,US,NY,M,3.0,ospC_AT12,12,L,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,UNY193,B477,US,NY,D,3.0,ospC_AT4,38,L,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,UNY203,MR616,US,NY,G,3.0,ospC_AT7,14,L,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,UWI247,MC104,US,WI,H,2.0,Hb,4,D,ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,UWI248,MC105,US,WI,H,2.0,Hb,32,L,ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,UWI263,MC123,US,WI,L,2.0,L1,29,ND,ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
282,UWI283,MC149,US,WI,O,3.0,Oa,221,L,ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
990,UNY990,B483,NY,NaN,N,2.0,NaN,NaN,ND,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
hybrid_paired = jupyter_df[jupyter_df['Rename_A'].isin(hybrid_df['paired_sample'])]
hybrid_paired = pandas.concat([hybrid_paired, (jupyter_df[jupyter_df['Rename_A'].isin(hybrid_paired['Rename_A'] + 'H')])])
hybrid_paired


,Rename_A,Original_Name,Origin,Location,OspC_Type,RST_Type,OspC_Allele,MLST,Disseminated,BC + (Y/N),...,Source.1,Code,Genus,Species.1,FastQC_1,BBDuk,FastQC_2,Spades,BBMerge,sp_identification
25,ESI26,B1904/13,EU,Slovenia,R,1.0,NaN,ST20,L,ND,...,Klemen,NaN,Borrelia,burgdorferi,y,y,y,y,y,y
28,UCT29,K1,US,CT,A,1.0,NaN,1,D,ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,UCT30,K11,US,CT,K,2.0,NaN,3,D,ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,UCT31,K10,US,CT,A,1.0,NaN,1,L,ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,UCT32,K15,US,CT,K,2.0,NaN,3,D,ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,URI112H,K75H,RI,NaN,NaN,2.0,NaN,NaN,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1193,UCT113H,K76H,CT,NaN,NaN,2.0,NaN,NaN,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1194,URI117H,K82H,RI,NaN,NaN,2.0,NaN,NaN,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1195,URI118H,K83H,RI,NaN,NaN,2.0,NaN,NaN,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
merged = pandas.concat([hybrid_paired, pacbio_paired])
merged

,Rename_A,Original_Name,Origin,Location,OspC_Type,RST_Type,OspC_Allele,MLST,Disseminated,BC + (Y/N),...,Source.1,Code,Genus,Species.1,FastQC_1,BBDuk,FastQC_2,Spades,BBMerge,sp_identification
25,ESI26,B1904/13,EU,Slovenia,R,1.0,NaN,ST20,L,ND,...,Klemen,NaN,Borrelia,burgdorferi,y,y,y,y,y,y
28,UCT29,K1,US,CT,A,1.0,NaN,1,D,ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,UCT30,K11,US,CT,K,2.0,NaN,3,D,ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,UCT31,K10,US,CT,A,1.0,NaN,1,L,ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,UCT32,K15,US,CT,K,2.0,NaN,3,D,ND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1212,UNY172P,B356P,NY,NaN,M,NaN,NaN,12,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,UNY193P,B477P,NY,NaN,D,NaN,NaN,38,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,UNY1038P,BL224P,NY,NaN,K,NaN,NaN,3,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,UNY208,MR641,US,NY,H,2.0,ospC_AT8,4,D,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
cols_to_keep = [ 'Rename_A', 'Original_Name', 'Origin', 'Location', 'OspC_Type',
                            'RST_Type', 'OspC_Allele', 'MLST', 'Disseminated','skin/blood_pairs',
                            'Original_Sample_Type','Genospecies', 'Host', 'Is_Male',
                            'Is_Female', 'Severity', 'Dx','Dissem_binary', 'ACA_binary',
                            'LNB_binary', 'Notes',]

In [15]:
merged = merged[cols_to_keep]

In [16]:
merged.loc[merged['Location'].isna(), 'Location'] = merged['Origin']
print(merged[merged['Location'] == 'Slovenia'])
merged.loc[merged['Location'] == 'Slovenia', 'Location'] = 'SI'
merged.loc[merged['Origin'] == 'Slovenia', 'Origin'] = 'EU'
print(merged[merged['Location'] == 'SI'])
merged.loc[merged['Origin'] == merged['Location'], 'Origin'] = 'US'
print(merged)


     Rename_A  Original_Name    Origin  Location OspC_Type  RST_Type  \
25      ESI26       B1904/13        EU  Slovenia         R       1.0   
1156   ESI26H  EU57_B190413H  Slovenia  Slovenia       NaN       NaN   

     OspC_Allele  MLST Disseminated skin/blood_pairs  ... Genospecies   Host  \
25           NaN  ST20            L              NaN  ...        EUBb  human   
1156         NaN   NaN          NaN              NaN  ...         NaN    NaN   

     Is_Male  Is_Female  Severity  Dx  Dissem_binary  ACA_binary  LNB_binary  \
25       NaN        NaN       NaN NaN            NaN         NaN         NaN   
1156     NaN        NaN       NaN NaN            NaN         NaN         NaN   

      Notes  
25      NaN  
1156    NaN  

[2 rows x 21 columns]
     Rename_A  Original_Name Origin Location OspC_Type  RST_Type OspC_Allele  \
25      ESI26       B1904/13     EU       SI         R       1.0         NaN   
1156   ESI26H  EU57_B190413H     EU       SI       NaN       NaN         NaN

In [17]:
origin_counts = merged['Origin'].value_counts()
location_counts = merged['Location'].value_counts()
print(f'origin:\n{origin_counts}')
print(f'location:\n{location_counts}')

origin:
US    110
EU      2
Name: Origin, dtype: int64
location:
RI    54
NY    26
CT    22
WI     8
SI     2
Name: Location, dtype: int64


In [18]:
merged['method'] = merged['Rename_A'].apply(lambda x: 'pacbio' if x.endswith('P') else ('Hybrid' if x.endswith('H') else 'illumina'))
merged

/var/folders/59/fs0fxd1x1wx1j_stw1ccjbcr0000gp/T/ipykernel_10263/3819204175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['method'] = merged['Rename_A'].apply(lambda x: 'pacbio' if x.endswith('P') else ('Hybrid' if x.endswith('H') else 'illumina'))


,Rename_A,Original_Name,Origin,Location,OspC_Type,RST_Type,OspC_Allele,MLST,Disseminated,skin/blood_pairs,...,Host,Is_Male,Is_Female,Severity,Dx,Dissem_binary,ACA_binary,LNB_binary,Notes,method
25,ESI26,B1904/13,EU,SI,R,1.0,NaN,ST20,L,NaN,...,human,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,illumina
28,UCT29,K1,US,CT,A,1.0,NaN,1,D,NaN,...,human,1.0,0.0,severe,NaN,1.0,NaN,NaN,NaN,illumina
29,UCT30,K11,US,CT,K,2.0,NaN,3,D,NaN,...,human,0.0,1.0,severe,NaN,1.0,NaN,NaN,NaN,illumina
30,UCT31,K10,US,CT,A,1.0,NaN,1,L,NaN,...,human,1.0,0.0,severe,NaN,1.0,NaN,NaN,NaN,illumina
31,UCT32,K15,US,CT,K,2.0,NaN,3,D,NaN,...,human,0.0,1.0,mild,NaN,1.0,NaN,NaN,NaN,illumina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1212,UNY172P,B356P,US,NY,M,NaN,NaN,12,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pacbio
1213,UNY193P,B477P,US,NY,D,NaN,NaN,38,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pacbio
1214,UNY1038P,BL224P,US,NY,K,NaN,NaN,3,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pacbio
207,UNY208,MR641,US,NY,H,2.0,ospC_AT8,4,D,BL546,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,illumina


In [19]:
print(len(pacbio_df) + len(hybrid_df))

66


In [20]:
not_in_illumina = merged[~merged['Rename_A'].isin(list_of_illumina)]
skipped_illumina_names = not_in_illumina[~not_in_illumina['Rename_A'].str.endswith(('P', 'H'))]['Rename_A'].tolist()
skipped_illumina_names


['UNY990', 'UNY1032', 'UNY1038', 'UNY1083', 'UNY1085', 'UNY1090', 'UNY1128']

### MISSING ILLUMINA ASSEMBLIES:
['UNY990', 'UNY1032', 'UNY1038', 'UNY1083', 'UNY1085', 'UNY1090', 'UNY1128']

## Ok now let's figure out the numbering part
mainly just to find where I need to start... 

In [21]:
scheme_key = pandas.read_csv('../old_lymeseq_tables/LymeSeq_SampleTrack-RenamingSchemeKey.csv')
scheme_key.shape


(387, 11)

In [22]:
scheme_key
final_scheme_key = scheme_key[['Rename_A3', 'Unnamed: 10']]
final_scheme_key = final_scheme_key.rename(columns={'Rename_A3': 'Original', 'Unnamed: 10': 'Renamed'})
#final_scheme_key.to_csv('final_scheme_key_v1.csv', index=False)

In [23]:
print(final_scheme_key)

    Original Renamed
0         N2     UM1
1         N4     UM2
2         N7     UM3
3         N9     UM4
4        N11     UM5
..       ...     ...
382    MC140   UW380
383    MC143   UW381
384    MC145   UW382
385    MC148   UW383
386    MC149   UW384

[387 rows x 2 columns]


In [24]:
list_of_countries = merged['Origin'].unique()
list_of_locations = merged['Location'].unique()
print(list_of_countries)
print(list_of_locations)

['EU' 'US']
['SI' 'CT' 'RI' 'NY' 'WI']


In [25]:
list_of_names = final_scheme_key['Renamed']
#print(list_of_names)
list_of_assemblies = merged['Rename_A']
print(list_of_assemblies)

25         ESI26
28         UCT29
29         UCT30
30         UCT31
31         UCT32
          ...   
1212     UNY172P
1213     UNY193P
1214    UNY1038P
207       UNY208
1209     UNY208P
Name: Rename_A, Length: 112, dtype: object


In [26]:
for asm in list_of_assemblies:
    if "_" in asm:
        print(asm)

In [27]:
import re
dirty_names = []
highest_ids = {}

for name in list_of_assemblies:
    if not any(location in name for location in list_of_locations):
        dirty_names.append(name)

for location in list_of_locations:
    max_id = 0
    for name in list_of_assemblies:
        #print(name)
        if location not in name:
            continue
        else:
            id_number = name[3:]
            #print(name,location,id_number)
            if id_number.isnumeric():
                id_number = int(id_number)
            else:
                id_number = re.sub(r'\D', '', id_number)
                id_number = int(id_number)
            if id_number > max_id:
                max_id = id_number
        highest_ids[location] = max_id

In [28]:
print(highest_ids)


{'SI': 26, 'CT': 113, 'RI': 120, 'NY': 1128, 'WI': 283}


In [29]:
location_counts = merged['Location'].value_counts()
print(location_counts)

RI    54
NY    26
CT    22
WI     8
SI     2
Name: Location, dtype: int64


## Now let's prep the renaming dataframes/dictionaries to make file management easier as we rename everything to play nicely with the metadata. 


In [30]:
# Add missing rows that somehow were left out of the table...
rename_pacbio_df = pacbio_df[['Isolate', 'paired_sample']]
missing_row = jupyter_df[jupyter_df['Original_Name'] == 'MR641']
missing_row = missing_row[['Rename_A', 'Original_Name']]
missing_row.rename(columns={'Rename_A': 'paired_sample' , 'Original_Name': 'Isolate'}, inplace=True)
rename_pacbio_df = pandas.concat([missing_row, rename_pacbio_df])
rename_pacbio_df

,paired_sample,Isolate
207,UNY208,MR641
0,UNY203,MR616
1,UWI283,MC149
2,UNY169,B348
3,B331P,B331
4,UNY1085,BL522
5,B418P,B418
6,UNY172,B356
7,UNY990,B483
8,UWI247,MC104


In [31]:
rename_hybrid_df = hybrid_df[['Isolate', 'paired_sample']]
rename_hybrid_df

,Isolate,paired_sample
19,EU57_B190413,ESI26
20,L113,ESI425
21,XD1,XYZ459
22,K21,UCT50
23,K83,URI118
24,K2,URI39
25,K19,URI48
26,K86,URI120
27,K22,URI34
28,K1,UCT29


In [32]:
pb_rename_dict = {}
for index, row in rename_pacbio_df.iterrows():
    sampleid = row['Isolate']
    rename = row['paired_sample']+'P'
    pb_rename_dict[sampleid] = rename

pb_rename_dict

{'MR641': 'UNY208P',
 'MR616': 'UNY203P',
 'MC149': 'UWI283P',
 'B348': 'UNY169P',
 'B331': 'B331PP',
 'BL522': 'UNY1085P',
 'B418': 'B418PP',
 'B356': 'UNY172P',
 'B483': 'UNY990P',
 'MC104': 'UWI247P',
 'B477': 'UNY193P',
 'B500': 'B500PP',
 'MC123': 'UWI263P',
 'MC105': 'UWI248P',
 'BL546': 'UNY1090P',
 'B247': 'UNY149P',
 'BL224': 'UNY1038P',
 'BL515': 'UNY1083P',
 'BL206': 'UNY1032P',
 'MR614': 'UNY1128P'}

In [33]:
hyb_rename_dict = {}
for index, row in rename_hybrid_df.iterrows():
    sampleid = row['Isolate'] + 'H'
    rename = row['paired_sample']+'H'
    hyb_rename_dict[sampleid] = rename

hyb_rename_dict


{'EU57_B190413H': 'ESI26H',
 'L113H': 'ESI425H',
 'XD1H': 'XYZ459H',
 'K21H': 'UCT50H',
 'K83H': 'URI118H',
 'K2H': 'URI39H',
 'K19H': 'URI48H',
 'K86H': 'URI120H',
 'K22H': 'URI34H',
 'K1H': 'UCT29H',
 'K64H': 'URI89H',
 'K56H': 'URI86H',
 'K13H': 'URI45H',
 'K24H': 'UCT35H',
 'K84H': 'UCT96H',
 'K6H': 'URI41H',
 'K25H': 'URI36H',
 'K73H': 'URI93H',
 'K75H': 'URI112H',
 'K58H': 'URI102H',
 'K90H': 'UCT124H',
 'K15H': 'UCT32H',
 'K74H': 'URI111H',
 'K62H': 'URI88H',
 'K66H': 'URI107H',
 'K10H': 'UCT31H',
 'K18H': 'URI47H',
 'K72H': 'UCT92H',
 'EU51_B192411H': 'ESI36H',
 'K30H': 'URI56H',
 'K12H': 'URI44H',
 'K7H': 'URI42H',
 'K80H': 'UCT95H',
 'K89H': 'K89HH',
 'K16H': 'URI33H',
 'EU14_B89613H': 'ESI403H',
 'K17H': 'URI46H',
 'K70H': 'UCT110H',
 'K57H': 'URI101H',
 'K76H': 'UCT113H',
 'K3H': 'URI40H',
 'K71H': 'URI91H',
 'K60H': 'URI87H',
 'K82H': 'URI117H',
 'K11H': 'UCT30H',
 'K59H': 'URI103H',
 'K69H': 'UCT109H'}

In [34]:
import pickle

with open('pb_rename_dict.pkl', 'wb') as f:
    pickle.dump(pb_rename_dict, f)

with open('hyb_rename_dict.pkl', 'wb') as f:
    pickle.dump(hyb_rename_dict, f)

In [35]:
merged[merged['Original_Name'].isin(['K1','K1H'])]

,Rename_A,Original_Name,Origin,Location,OspC_Type,RST_Type,OspC_Allele,MLST,Disseminated,skin/blood_pairs,...,Host,Is_Male,Is_Female,Severity,Dx,Dissem_binary,ACA_binary,LNB_binary,Notes,method
28,UCT29,K1,US,CT,A,1.0,NaN,1,D,NaN,...,human,1.0,0.0,severe,NaN,1.0,NaN,NaN,NaN,illumina
1157,UCT29H,K1H,US,CT,NaN,1.0,NaN,NaN,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hybrid


## RENAME HYBRID ASSEMBLY FILES AND ANNOTATIONS

### Rename contigs to the correct name for lookup table renaming.

In [36]:
# lets try and rename all of the files given the tables from above...
for filepath in glob.glob(f'{assembly_dirs}/hybrid/contigs/*.fasta'):
    print(filepath)
    name = filepath.split('.')[0]
    filename = f'{name}H.fasta'
    print(filename)
    #os.rename(filepath,filename)

/Users/mf019/bioinformatics/longread_GWAS/assemblies/hybrid/contigs/URI40H.fasta
/Users/mf019/bioinformatics/longread_GWAS/assemblies/hybrid/contigs/URI40HH.fasta
/Users/mf019/bioinformatics/longread_GWAS/assemblies/hybrid/contigs/ESI403H.fasta
/Users/mf019/bioinformatics/longread_GWAS/assemblies/hybrid/contigs/ESI403HH.fasta
/Users/mf019/bioinformatics/longread_GWAS/assemblies/hybrid/contigs/UCT95H.fasta
/Users/mf019/bioinformatics/longread_GWAS/assemblies/hybrid/contigs/UCT95HH.fasta
/Users/mf019/bioinformatics/longread_GWAS/assemblies/hybrid/contigs/K89HH.fasta
/Users/mf019/bioinformatics/longread_GWAS/assemblies/hybrid/contigs/K89HHH.fasta
/Users/mf019/bioinformatics/longread_GWAS/assemblies/hybrid/contigs/UCT50H.fasta
/Users/mf019/bioinformatics/longread_GWAS/assemblies/hybrid/contigs/UCT50HH.fasta
/Users/mf019/bioinformatics/longread_GWAS/assemblies/hybrid/contigs/UCT110H.fasta
/Users/mf019/bioinformatics/longread_GWAS/assemblies/hybrid/contigs/UCT110HH.fasta
/Users/mf019/bioinfo

In [37]:
# OK so if you accidentally add one too many H's to the end of the filename, you can use this to fix it
##for filepath in glob.glob(f'{assembly_dirs}/hybrid/contigs/*.fasta'):
##    print(f'old: {filepath}')
##    name = filepath.split('.')[0][:-1:]
##    filename = f'{name}.fasta'
##    print(f'new: {filename}')
##    #os.rename(filepath,filename)

In [38]:
files = glob.glob(f'{assembly_dirs}/hybrid/contigs/*.fasta')

for filepath in files:
    name = filepath.split('/')[-1].split('.')[0]
    newname = f'{hyb_rename_dict[name]}'
    oldname = filepath.split('/')[-1]
    oldpath = '/'.join(filepath.split('/')[:-1:]) #+ f'/{newname}'#split('.')[0]
    newpath = f'{oldpath}'
    extension = filepath.split('.')[-1]
    rename_file = f'{newpath}/{newname}.{extension}'
    print(f'stripped name {name}')
    print(f'original file path {filepath}/')
    print(f'rename file first: {rename_file}\n')
    #os.rename(filepath,rename_file) # rename files

KeyError: 'URI40H'

### Now let's rename the annotations

In [39]:
dirs = glob.glob(f'{assembly_dirs}/hybrid/annotation/*')

for directory in dirs:
    print(directory)
    name = directory.split('/')[-1]
    print(f'stripped name {name}')
    oldname = directory.split('/')[-1]
    print(f'old name {oldname}')
    newname = f'{hyb_rename_dict[name]}'
    print(f'new name {newname}')
    oldpath = '/'.join(directory.split('/')[:-1:]) + f'/{oldname}'#split('.')[0]
    newpath = '/'.join(directory.split('/')[:-1:]) + f'/{newname}/'#split('.')[0]
    print(f'old path {oldpath}/')
    print(f'new path {newpath}\n')
    #os.rename(oldpath,newpath)

files = glob.glob(f'{assembly_dirs}/hybrid/annotation/*/*')

for filepath in files:
    name = filepath.split('/')[-1].split('.')[0]
    newname = f'{hyb_rename_dict[name]}'
    oldname = filepath.split('/')[-1]
    oldpath = '/'.join(filepath.split('/')[:-2:]) + f'/{newname}'#split('.')[0]
    newpath = f'{oldpath}'
    extension = '.'.join(filepath.split('.')[1::])
    rename_file = f'{newpath}/{newname}.{extension}'
    print(f'stripped name {name}')
    print(f'original file path {filepath}/')
    print(f'rename file first: {rename_file}')
    #os.rename(filepath,rename_file) # rename files

/Users/mf019/bioinformatics/longread_GWAS/assemblies/hybrid/annotation/UCT29H
stripped name UCT29H
old name UCT29H


KeyError: 'UCT29H'

## RENAME PACBIO ASSEMBLY FILES AND ANNOTATIONS
### Okay, so for some reason I am missing metadata for one pacbio assembly. 
error given: `KeyError: 'MR641'`
I will briefly add this entry to the bottom of my merged dataframe and also to my dictionary for renaming.
I will add these entries above when I am building the merged dataframe.


In [40]:
dirs = glob.glob(f'{assembly_dirs}/pacbio/annotation/*')

for directory in dirs:
    print(directory)
    name = directory.split('/')[-1]
    print(f'stripped name {name}')
    oldname = directory.split('/')[-1]
    print(f'old name {oldname}')
    newname = f'{pb_rename_dict[name]}'
    print(f'new name {newname}')
    oldpath = '/'.join(directory.split('/')[:-1:]) + f'/{oldname}'#split('.')[0]
    newpath = '/'.join(directory.split('/')[:-1:]) + f'/{newname}/'#split('.')[0]
    print(f'old path {oldpath}/')
    print(f'new path {newpath}\n')
    #os.rename(oldpath,newpath)

files = glob.glob(f'{assembly_dirs}/pacbio/annotation/*/*')

for filepath in files:
    name = filepath.split('/')[-1].split('.')[0]
    newname = f'{pb_rename_dict[name]}'
    oldname = filepath.split('/')[-1]
    oldpath = '/'.join(filepath.split('/')[:-2:]) + f'/{newname}'#split('.')[0]
    newpath = f'{oldpath}'
    extension = '.'.join(filepath.split('.')[1::])
    rename_file = f'{newpath}/{newname}.{extension}'
    print(f'stripped name {name}')
    print(f'original file path {filepath}/')
    print(f'rename file first: {rename_file}')
    #os.rename(filepath,rename_file) # rename files


/Users/mf019/bioinformatics/longread_GWAS/assemblies/pacbio/annotation/B418PP
stripped name B418PP
old name B418PP


KeyError: 'B418PP'

### Rename Contigs files

In [111]:
# Rename the contigs for use with lookup table!
for filepath in glob.glob(f'{assembly_dirs}/pacbio/contigs/*.fasta'):
    print(f'old: {filepath}')
    name = filepath.split('.')[0]
    filename = f'{name}P.fasta'
    print(f'new: {filename}')
    #os.rename(filepath,filename)


old: /Users/mf019/bioinformatics/longread_GWAS/assemblies/pacbio/contigs/UNY1083P.fasta
new: /Users/mf019/bioinformatics/longread_GWAS/assemblies/pacbio/contigs/UNY1083PP.fasta
old: /Users/mf019/bioinformatics/longread_GWAS/assemblies/pacbio/contigs/UNY169P.fasta
new: /Users/mf019/bioinformatics/longread_GWAS/assemblies/pacbio/contigs/UNY169PP.fasta
old: /Users/mf019/bioinformatics/longread_GWAS/assemblies/pacbio/contigs/UWI247P.fasta
new: /Users/mf019/bioinformatics/longread_GWAS/assemblies/pacbio/contigs/UWI247PP.fasta
old: /Users/mf019/bioinformatics/longread_GWAS/assemblies/pacbio/contigs/UNY1085P.fasta
new: /Users/mf019/bioinformatics/longread_GWAS/assemblies/pacbio/contigs/UNY1085PP.fasta
old: /Users/mf019/bioinformatics/longread_GWAS/assemblies/pacbio/contigs/B331PP.fasta
new: /Users/mf019/bioinformatics/longread_GWAS/assemblies/pacbio/contigs/B331PPP.fasta
old: /Users/mf019/bioinformatics/longread_GWAS/assemblies/pacbio/contigs/UWI263P.fasta
new: /Users/mf019/bioinformatics/lon

In [112]:
files = glob.glob(f'{assembly_dirs}/pacbio/contigs/*.fasta')

for filepath in files:
    name = filepath.split('/')[-1].split('.')[0]
    newname = f'{pb_rename_dict[name]}'
    oldname = filepath.split('/')[-1]
    oldpath = '/'.join(filepath.split('/')[:-1:]) #+ f'/{newname}'#split('.')[0]
    newpath = f'{oldpath}'
    extension = '.'.join(filepath.split('.')[1::])
    rename_file = f'{newpath}/{newname}.{extension}'
    print(f'stripped name {name}')
    print(f'original file path {filepath}/')
    print(f'rename file first: {rename_file}\n')
    #os.rename(filepath,rename_file) # rename files

KeyError: 'UNY1083P'

### Rename Illumina Files (just drop the _200)

In [113]:
import glob
files = glob.glob(f'{assembly_dirs}/illumina/contigs/*.fasta')

for filepath in files:
    name = filepath.split('/')[-1].split('.')[0]
    newname = f'{name.replace("_200", "")}'
    oldname = filepath.split('/')[-1]
    oldpath = '/'.join(filepath.split('/')[:-1:]) #+ f'/{newname}'#split('.')[0]
    rename_file = f'{oldpath}/{newname}.fasta'
    print(f'stripped name {name}')
    print(f'original file path {filepath}/')
    print(f'rename file first: {rename_file}\n')
    #os.rename(filepath,rename_file) # rename files

stripped name UNY210
original file path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/contigs/UNY210.fasta/
rename file first: /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/contigs/UNY210.fasta

stripped name UNY154
original file path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/contigs/UNY154.fasta/
rename file first: /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/contigs/UNY154.fasta

stripped name ESI318
original file path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/contigs/ESI318.fasta/
rename file first: /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/contigs/ESI318.fasta

stripped name UWI241
original file path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/contigs/UWI241.fasta/
rename file first: /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/contigs/UWI241.fasta

stripped name UWI281
original file path /Users/mf019/bioinformatics/longread_GWAS/assemblies

In [114]:
dirs = glob.glob(f'{assembly_dirs}/illumina/annotation/*')

for directory in dirs:
    print(directory)
    name = directory.split('/')[-1]
    print(f'stripped name {name}')
    oldname = directory.split('/')[-1]
    print(f'old name {oldname}')
    newname = name.replace('_200','')
    print(f'new name {newname}')
    oldpath = '/'.join(directory.split('/')[:-1:]) + f'/{oldname}'#split('.')[0]
    newpath = '/'.join(directory.split('/')[:-1:]) + f'/{newname}/'#split('.')[0]
    print(f'old path {oldpath}/')
    print(f'new path {newpath}\n')
    #os.rename(oldpath,newpath)


/Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/ESI312
stripped name ESI312
old name ESI312
new name ESI312
old path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/ESI312/
new path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/ESI312/

/Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/URI44
stripped name URI44
old name URI44
new name URI44
old path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/URI44/
new path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/URI44/

/Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/UNY136
stripped name UNY136
old name UNY136
new name UNY136
old path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/UNY136/
new path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/UNY136/

/Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/an

In [115]:
files = glob.glob(f'{assembly_dirs}/illumina/annotation/*/*')

for filepath in files:
    name = filepath.split('/')[-1].split('.')[0]
    newname = name.replace('_200','')
    oldname = filepath.split('/')[-1]
    oldpath = '/'.join(filepath.split('/')[:-2:]) + f'/{newname}'#split('.')[0]
    newpath = f'{oldpath}'
    extension = '.'.join(filepath.split('.')[1::])
    rename_file = f'{newpath}/{newname}.{extension}'
    print(f'stripped name {name}')
    print(f'original file path {filepath}/')
    print(f'rename file first: {rename_file}')
    #os.rename(filepath,rename_file) # rename files


stripped name ESI312
original file path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/ESI312/ESI312.json/
rename file first: /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/ESI312/ESI312.json
stripped name ESI312
original file path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/ESI312/ESI312.embl/
rename file first: /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/ESI312/ESI312.embl
stripped name ESI312
original file path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/ESI312/ESI312.ffn/
rename file first: /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/ESI312/ESI312.ffn
stripped name ESI312
original file path /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/ESI312/ESI312.log/
rename file first: /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/ESI312/ESI312.log
stripped name ESI312
origina

In [116]:
merged = merged.sort_values('Rename_A')
merged = merged.reset_index(drop=True, inplace=False)
merged.rename(columns={'Rename_A': 'Isolate'}, inplace=True)

In [117]:
print(jupyter_df[jupyter_df['Original_Name'] == 'BL546']) # ok this is paired with MR641

     Rename_A Original_Name Origin Location OspC_Type  RST_Type OspC_Allele  \
1090  UNY1090         BL546     NY      NaN         H       2.0         NaN   

     MLST Disseminated BC + (Y/N)  ... Source.1 Code Genus Species.1  \
1090    4            D        NaN  ...      NaN  NaN   NaN       NaN   

      FastQC_1  BBDuk  FastQC_2 Spades BBMerge sp_identification  
1090       NaN    NaN       NaN    NaN     NaN               NaN  

[1 rows x 54 columns]


In [118]:
# How did a newline get into my data? let's just fix that.
merged.replace(r'\n', '', regex=True, inplace=True)
merged.loc[merged['Original_Name'] == 'B348', 'skin/blood_pairs']


38    BL202
Name: skin/blood_pairs, dtype: object

In [119]:
#merged.to_csv(v7_file, index=False, sep=',')

In [42]:
paired_outdir = f'{assembly_dirs}/paired_assemblies'
# let's move all of the files to a separate directory for paired use/ez download
#os.mkdir(paired_outdir)

In [121]:
#import shutil

#list_of_assemblies = merged['Isolate'].to_list()
#for asm in list_of_assemblies:
#    for filepath in glob.glob(f'{assembly_dirs}/*/*/{asm}.fasta'):
#        print(f'OG: {filepath}')
#        filename = filepath.split('/')[-1]
#        newpath = f'{paired_outdir}/{filename}'
#        print(f'new: {newpath}')
#        #shutil.copy(filepath,newpath)

In [122]:
#for asm in list_of_assemblies:
#    for filepath in glob.glob(f'{assembly_dirs}/*/*/{asm}/{asm}.gff3'):
#        print(f'OG: {filepath}')
#        filename = filepath.split('/')[-1]
#        newpath = f'{paired_outdir}/annotation/{filename}'
#        print(f'new: {newpath}')
#        #shutil.copy(filepath,newpath)

In [48]:
for asm in list_of_assemblies:
    for filepath in glob.glob(f'{assembly_dirs}/*/*/{asm}/{asm}.gbff'):
        print(f'OG: {filepath}')
        filename = filepath.split('/')[-1]
        newpath = f'{paired_outdir}/annotation/genbank/{filename}'
        print(f'new: {newpath}')
        shutil.copy(filepath,newpath)

OG: /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/ESI26/ESI26.gbff
new: /Users/mf019/bioinformatics/longread_GWAS/assemblies/paired_assemblies/annotation/genbank/ESI26.gbff
OG: /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/UCT29/UCT29.gbff
new: /Users/mf019/bioinformatics/longread_GWAS/assemblies/paired_assemblies/annotation/genbank/UCT29.gbff
OG: /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/UCT30/UCT30.gbff
new: /Users/mf019/bioinformatics/longread_GWAS/assemblies/paired_assemblies/annotation/genbank/UCT30.gbff
OG: /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/UCT31/UCT31.gbff
new: /Users/mf019/bioinformatics/longread_GWAS/assemblies/paired_assemblies/annotation/genbank/UCT31.gbff
OG: /Users/mf019/bioinformatics/longread_GWAS/assemblies/illumina/annotation/UCT32/UCT32.gbff
new: /Users/mf019/bioinformatics/longread_GWAS/assemblies/paired_assemblies/annotation/genbank/UCT32.gbff


In [135]:
# I need to merge the old metadata (primarily MLST + OspC) with the new metadata
merged

,Isolate,Original_Name,Origin,Location,OspC_Type,RST_Type,OspC_Allele,MLST,Disseminated,skin/blood_pairs,...,Host,Is_Male,Is_Female,Severity,Dx,Dissem_binary,ACA_binary,LNB_binary,Notes,method
0,ESI26,B1904/13,EU,SI,R,1.0,NaN,ST20,L,NaN,...,human,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,illumina
1,ESI26H,EU57_B190413H,EU,SI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hybrid
2,UCT109,K69,US,CT,A,1.0,NaN,1,L,NaN,...,human,1.0,0.0,mild,NaN,0.0,NaN,NaN,NaN,illumina
3,UCT109H,K69H,US,CT,NaN,1.0,NaN,NaN,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hybrid
4,UCT110,K70,US,CT,K,2.0,NaN,3,D,NaN,...,human,1.0,0.0,moderate,NaN,1.0,NaN,NaN,NaN,illumina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,UWI248P,MC105P,US,WI,H,NaN,NaN,31,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pacbio
108,UWI263,MC123,US,WI,L,2.0,L1,29,ND,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,illumina
109,UWI263P,MC123P,US,WI,L,NaN,NaN,29,ND,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pacbio
110,UWI283,MC149,US,WI,O,3.0,Oa,221,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,illumina


In [136]:
merged.keys()

Index(['Isolate', 'Original_Name', 'Origin', 'Location', 'OspC_Type',
       'RST_Type', 'OspC_Allele', 'MLST', 'Disseminated', 'skin/blood_pairs',
       'Original_Sample_Type', 'Genospecies', 'Host', 'Is_Male', 'Is_Female',
       'Severity', 'Dx', 'Dissem_binary', 'ACA_binary', 'LNB_binary', 'Notes',
       'method'],
      dtype='object')

In [210]:
filtered_merged = merged.drop(columns=['OspC_Type', 'RST_Type', 'MLST', 'Is_Male', 'Is_Female', 'Dissem_binary', 'ACA_binary', 'LNB_binary', 'Notes', 'Dx', 'Original_Sample_Type', 'Genospecies', 'Host'])
filtered_merged['Original_Name'] = filtered_merged['Original_Name'].apply(lambda x: x[:-1] if x.endswith(('P', 'H')) else x)


In [211]:
filtered_merged.value_counts(subset=['method'])
filtered_merged

,Isolate,Original_Name,Origin,Location,OspC_Allele,Disseminated,skin/blood_pairs,Severity,method
0,ESI26,B1904/13,EU,SI,NaN,L,NaN,NaN,illumina
1,ESI26H,EU57_B190413,EU,SI,NaN,NaN,NaN,NaN,Hybrid
2,UCT109,K69,US,CT,NaN,L,NaN,mild,illumina
3,UCT109H,K69,US,CT,NaN,L,NaN,NaN,Hybrid
4,UCT110,K70,US,CT,NaN,D,NaN,moderate,illumina
...,...,...,...,...,...,...,...,...,...
107,UWI248P,MC105,US,WI,NaN,L,NaN,NaN,pacbio
108,UWI263,MC123,US,WI,L1,ND,NaN,NaN,illumina
109,UWI263P,MC123,US,WI,NaN,ND,NaN,NaN,pacbio
110,UWI283,MC149,US,WI,Oa,L,NaN,NaN,illumina


In [203]:
v6_df

,Isolate,MLST,RST_Type,Source,Origin,Location,skin/blood_pairs,Disseminated,method,OspC_Type,AT_type,newname,paired_sample
0,MR616,14,3.0,Skin,US,NY,NaN,L,pacbio,G,AT7,MR616P,UNY203
1,MC149,221,NaN,Skin,US,WI,NaN,L,pacbio,O,AT3,MC149P,UWI283
2,B348,19,3.0,Skin,US,NY,NaN,D,pacbio,E,AT5,B348P,UNY169
3,B331,16,3.0,Skin,US,NY,NaN,L,pacbio,I,AT9,B331P,NaN
4,BL522,12,3.0,NaN,NY,NaN,NaN,D,pacbio,M,AT12,BL522P,UNY1085
...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,URI19,59,1.0,NaN,US,RI,NaN,L,illumina,B,AT2,NaN,NaN
361,UMA13,59,1.0,NaN,US,Nantucket,NaN,L,illumina,B,NaN,NaN,NaN
362,UMA2,8,2.0,NaN,US,Nantucket,NaN,L,illumina,F,AT6,NaN,NaN
363,URI84,59,2.0,NaN,US,RI,NaN,L,illumina,B,NaN,NaN,NaN


In [212]:
v6_nonlumina = v6_df[v6_df['method'] != 'illumina']
v6_lumina = v6_df[v6_df['method'] == 'illumina']
print(v6_nonlumina.keys())
print(v6_lumina.keys())

Index(['Isolate', 'MLST', 'RST_Type', 'Source', 'Origin', 'Location',
       'skin/blood_pairs', 'Disseminated', 'method', 'OspC_Type', 'AT_type',
       'newname', 'paired_sample'],
      dtype='object')
Index(['Isolate', 'MLST', 'RST_Type', 'Source', 'Origin', 'Location',
       'skin/blood_pairs', 'Disseminated', 'method', 'OspC_Type', 'AT_type',
       'newname', 'paired_sample'],
      dtype='object')


In [213]:
v6_lumina

,Isolate,MLST,RST_Type,Source,Origin,Location,skin/blood_pairs,Disseminated,method,OspC_Type,AT_type,newname,paired_sample
66,UNY147,34,3.0,NaN,US,NY,NaN,L,illumina,J,NaN,NaN,NaN
67,UNY218,37,3.0,NaN,US,NY,NaN,L,illumina,T,AT14,NaN,NaN
68,UNY201,34,3.0,NaN,US,NY,NaN,D,illumina,J,AT10,NaN,NaN
69,UWI272,55,1.0,NaN,US,WI,NaN,D,illumina,A,AT1,NaN,NaN
70,UNY210,18,3.0,NaN,US,NY,NaN,L,illumina,U,AT15,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,URI19,59,1.0,NaN,US,RI,NaN,L,illumina,B,AT2,NaN,NaN
361,UMA13,59,1.0,NaN,US,Nantucket,NaN,L,illumina,B,NaN,NaN,NaN
362,UMA2,8,2.0,NaN,US,Nantucket,NaN,L,illumina,F,AT6,NaN,NaN
363,URI84,59,2.0,NaN,US,RI,NaN,L,illumina,B,NaN,NaN,NaN


In [214]:
v6_nonlumina['paired_sample'] = v6_nonlumina['paired_sample'].astype(str)
v6_nonlumina['method'] = v6_nonlumina['method'].astype(str)
v6_nonlumina['newname_2'] = v6_nonlumina.apply(lambda row: row['paired_sample'] + str('P') if row['method'] == 'pacbio' else (row['paired_sample'] + str('H') if row['method'] == 'hybrid' else str('')), axis=1)

/var/folders/59/fs0fxd1x1wx1j_stw1ccjbcr0000gp/T/ipykernel_50703/2540739463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  v6_nonlumina['paired_sample'] = v6_nonlumina['paired_sample'].astype(str)
/var/folders/59/fs0fxd1x1wx1j_stw1ccjbcr0000gp/T/ipykernel_50703/2540739463.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  v6_nonlumina['method'] = v6_nonlumina['method'].astype(str)
/var/folders/59/fs0fxd1x1wx1j_stw1ccjbcr0000gp/T/ipykernel_50703/2540739463.py:3: SettingWithCopyWarning: 
A value is tryi

In [215]:
v6_lumina

,Isolate,MLST,RST_Type,Source,Origin,Location,skin/blood_pairs,Disseminated,method,OspC_Type,AT_type,newname,paired_sample
66,UNY147,34,3.0,NaN,US,NY,NaN,L,illumina,J,NaN,NaN,NaN
67,UNY218,37,3.0,NaN,US,NY,NaN,L,illumina,T,AT14,NaN,NaN
68,UNY201,34,3.0,NaN,US,NY,NaN,D,illumina,J,AT10,NaN,NaN
69,UWI272,55,1.0,NaN,US,WI,NaN,D,illumina,A,AT1,NaN,NaN
70,UNY210,18,3.0,NaN,US,NY,NaN,L,illumina,U,AT15,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,URI19,59,1.0,NaN,US,RI,NaN,L,illumina,B,AT2,NaN,NaN
361,UMA13,59,1.0,NaN,US,Nantucket,NaN,L,illumina,B,NaN,NaN,NaN
362,UMA2,8,2.0,NaN,US,Nantucket,NaN,L,illumina,F,AT6,NaN,NaN
363,URI84,59,2.0,NaN,US,RI,NaN,L,illumina,B,NaN,NaN,NaN


In [216]:

filtered_v6_il = v6_lumina.drop(columns=['Origin', 'Location','Disseminated', 'method','skin/blood_pairs', 'newname', 'paired_sample'])
filtered_v6_nil = v6_nonlumina.drop(columns=['Isolate','Origin', 'Location','Disseminated', 'method','skin/blood_pairs'])
print(filtered_v6_il)
print(filtered_v6_nil)

    Isolate MLST  RST_Type Source OspC_Type AT_type
66   UNY147   34       3.0    NaN         J     NaN
67   UNY218   37       3.0    NaN         T    AT14
68   UNY201   34       3.0    NaN         J    AT10
69   UWI272   55       1.0    NaN         A     AT1
70   UNY210   18       3.0    NaN         U    AT15
..      ...  ...       ...    ...       ...     ...
360   URI19   59       1.0    NaN         B     AT2
361   UMA13   59       1.0    NaN         B     NaN
362    UMA2    8       2.0    NaN         F     AT6
363   URI84   59       2.0    NaN         B     NaN
364   URI20   14       3.0    NaN         G     AT7

[299 rows x 6 columns]
   MLST  RST_Type Source OspC_Type AT_type newname paired_sample newname_2
0    14       3.0   Skin         G     AT7  MR616P        UNY203   UNY203P
1   221       NaN   Skin         O     AT3  MC149P        UWI283   UWI283P
2    19       3.0   Skin         E     AT5   B348P        UNY169   UNY169P
3    16       3.0   Skin         I     AT9   B331P  

In [265]:
v8_merged_nil = filtered_merged.merge(filtered_v6_nil, left_on='Isolate', right_on='newname_2', how='left')
v8_merged_nil = v8_merged_nil[v8_merged_nil['Isolate'].str.endswith(('H', 'P'))]
v8_merged_nil


,Isolate,Original_Name,Origin,Location,OspC_Allele,Disseminated,skin/blood_pairs,Severity,method,MLST,RST_Type,Source,OspC_Type,AT_type,newname,paired_sample,newname_2
1,ESI26H,EU57_B190413,EU,SI,NaN,NaN,NaN,NaN,Hybrid,20,NaN,NaN,M,AT7,EU57_B190413H,ESI26,ESI26H
3,UCT109H,K69,US,CT,NaN,L,NaN,NaN,Hybrid,1,1.0,NaN,A,AT1,K69H,UCT109,UCT109H
5,UCT110H,K70,US,CT,NaN,D,NaN,NaN,Hybrid,3,2.0,NaN,K,AT11,K70H,UCT110,UCT110H
7,UCT113H,K76,US,CT,NaN,D,NaN,NaN,Hybrid,4,2.0,NaN,H,AT8,K76H,UCT113,UCT113H
9,UCT29H,K1,US,CT,NaN,D,NaN,NaN,Hybrid,1,1.0,NaN,A,AT1,K1H,UCT29,UCT29H
11,UCT30H,K11,US,CT,NaN,D,NaN,NaN,Hybrid,3,2.0,NaN,K,AT11,K11H,UCT30,UCT30H
13,UCT31H,K10,US,CT,NaN,L,NaN,NaN,Hybrid,1,1.0,NaN,A,AT1,K10H,UCT31,UCT31H
15,UCT32H,K15,US,CT,NaN,D,NaN,NaN,Hybrid,3,2.0,NaN,K,AT11,K15H,UCT32,UCT32H
17,UCT35H,K24,US,CT,NaN,L,NaN,NaN,Hybrid,14,3.0,NaN,G,AT7,K24H,UCT35,UCT35H
19,UCT50H,K21,US,CT,NaN,D,NaN,NaN,Hybrid,19,3.0,NaN,E,AT5,K21H,UCT50,UCT50H


In [266]:
v8_merged_il = filtered_merged.merge(filtered_v6_il, left_on='Isolate', right_on='Isolate', how='left')
v8_merged_il = v8_merged_il[~v8_merged_il['Isolate'].str.endswith(('H', 'P'))]

In [267]:
v8_metadata = pandas.concat([v8_merged_nil, v8_merged_il])

In [313]:
v8_metadata.sort_values('Isolate', inplace=True)
v8_metadata.reset_index(drop=True, inplace=True)
#v8_metadata = v8_metadata.drop(columns=['newname_2', 'newname', 'paired_sample'])
copy_v8 = v8_metadata.copy()

In [314]:
for index, row in copy_v8.iterrows():
    if row['Isolate'].endswith(('H', 'P')):
        if pandas.isnull(row['MLST']):
            copy_v8.loc[index, 'MLST'] = copy_v8.loc[index-1, 'MLST']
        if pandas.isnull(row['OspC_Type']):
            copy_v8.loc[index, 'OspC_Type'] = copy_v8.loc[index-1, 'OspC_Type']
        if pandas.isnull(row['RST_Type']):
            copy_v8.loc[index, 'RST_Type'] = copy_v8.loc[index-1, 'RST_Type']
        if pandas.isnull(row['AT_type']):
            copy_v8.loc[index, 'AT_type'] = copy_v8.loc[index-1, 'AT_type']
        if pandas.isnull(row['Disseminated']):
            copy_v8.loc[index, 'Disseminated'] = copy_v8.loc[index-1, 'Disseminated']
        if pandas.isnull(row['Severity']):
            copy_v8.loc[index, 'Severity'] = copy_v8.loc[index-1, 'Severity']
    else:
        continue


In [316]:
#copy_v8 = copy_v8.drop(columns=['OspC_Allele'])
#copy_v8['Source'] = copy_v8['Source']

,Isolate,Original_Name,Origin,Location,OspC_Allele,Disseminated,skin/blood_pairs,Severity,method,MLST,RST_Type,Source,OspC_Type,AT_type
0,ESI26,B1904/13,EU,SI,NaN,L,NaN,NaN,illumina,20,1.0,NaN,R,AT7
1,ESI26H,EU57_B190413,EU,SI,NaN,L,NaN,NaN,Hybrid,20,1.0,NaN,M,AT7
2,UCT109,K69,US,CT,NaN,L,NaN,mild,illumina,1,1.0,NaN,A,AT1
3,UCT109H,K69,US,CT,NaN,L,NaN,mild,Hybrid,1,1.0,NaN,A,AT1
4,UCT110,K70,US,CT,NaN,D,NaN,moderate,illumina,3,2.0,NaN,K,AT11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,UWI248P,MC105,US,WI,NaN,L,NaN,NaN,pacbio,32,2.0,Skin,H,AT8
108,UWI263,MC123,US,WI,L1,ND,NaN,NaN,illumina,29,2.0,NaN,L,AT10
109,UWI263P,MC123,US,WI,NaN,ND,NaN,NaN,pacbio,29,2.0,Skin,L,AT10
110,UWI283,MC149,US,WI,Oa,L,NaN,NaN,illumina,221,3.0,NaN,O,AT3


In [317]:
pairs = list(copy_v8['skin/blood_pairs'][copy_v8['skin/blood_pairs'].notnull()])
print(pairs)


['BL146', 'BL202', 'BL546']


In [320]:

for index, row in copy_v8.iterrows():
    if row['Original_Name'] in pairs:
        print(row['Original_Name'])
        copy_v8.loc[index, 'Source'] = 'Blood'
    else:
        if pandas.isnull(row['Source']):
            copy_v8.loc[index, 'Source'] = 'Skin'
        continue

BL546
BL546


In [321]:
copy_v8['Source'][copy_v8['Isolate'] == 'UWI247'] = 'CSF'
copy_v8['Source'].value_counts()

/var/folders/59/fs0fxd1x1wx1j_stw1ccjbcr0000gp/T/ipykernel_50703/604794693.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_v8['Source'][copy_v8['Isolate'] == 'UWI247'] = 'CSF'


Skin     108
Blood      2
CSF        2
Name: Source, dtype: int64

In [324]:
copy_v8 = copy_v8.drop(columns=['OspC_Allele'])

In [325]:
copy_v8.to_csv('../wrangled/longread_metadata_v8.csv', index=False)